In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
cv2.VideoCapture(0)

< cv2.VideoCapture 000002845AA36590>

In [ ]:
import cv2
import numpy as np
import json
# Load the known faces and names from the JSON file
with open("known_faces.json", "r") as f:  # Replace with the path to your JSON file
    data = json.load(f)
#Initialize lists for face encodings and names
known_face_encodings = []
known_face_names = []
# Initialize the face recognizer (LBPH)
recognizer = cv2.face.LBPHFaceRecognizer_create()
# Load images and their names from the JSON data
for person in data["known_faces"]:
    image_path = person["image_path"]
    name = person["name"]
    # Load the image of the known person
    person_image = cv2.imread(image_path)
    # Convert image to grayscale (LBPH recognizer works with grayscale images)
    gray_image = cv2.cvtColor(person_image, cv2.COLOR_BGR2GRAY)
    # Initialize face detector (Haar Cascade for face detection)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    # Detect faces in the image
    faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5)
    # Loop over detected faces and extract encoding (just taking the first face)
    for (x, y, w, h) in faces:
        face_image = gray_image[y:y+h, x:x+w]
        known_face_encodings.append(face_image)  # Add the face region as an encoding
        known_face_names.append(name)  # Store the name
# Initialize the video capture (0 for webcam)
video_capture = cv2.VideoCapture(0)
# Train the recognizer with the known faces and their names
recognizer.train(known_face_encodings, np.array([i for i in range(len(known_face_names))]))
while True:
    # Capture a frame from the webcam
    ret, frame = video_capture.read()
    # Convert the frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5)
    # Loop over each face found in the frame
    for (x, y, w, h) in faces:
        face_image = gray_frame[y:y+h, x:x+w]
        # Recognize the face
        label, confidence = recognizer.predict(face_image)
        # Get the name of the recognized person
        name = known_face_names[label] if confidence < 100 else "Unknown"  # Label is index, confidence is the distance
        # Draw a rectangle around the face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
        # Put the name of the recognized person above the face
        cv2.putText(frame, name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    # Display the resulting frame with recognized faces
    cv2.imshow('Video', frame)
    # Break the loop if the user presses the 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# Release the video capture object and close any OpenCV windows
video_capture.release()
cv2.destroyAllWindows()
